# Emergency Detection Model Training
## Large Events Emergency Management System

This notebook trains ML models for detecting emergencies (fire, medical, security) using Google Colab's GPU resources.


## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install tensorflow==2.13.0
!pip install torch torchvision
!pip install opencv-python
!pip install scikit-learn
!pip install pandas numpy matplotlib seaborn
!pip install pillow
!pip install tqdm

# Check GPU availability
import tensorflow as tf
import torch
print(f"TensorFlow version: {tf.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"GPU available (TF): {tf.config.list_physical_devices('GPU')}")
print(f"GPU available (PyTorch): {torch.cuda.is_available()}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
import os
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
torch.manual_seed(42)

print("Libraries imported successfully!")

## 2. Data Preparation and Synthetic Data Generation

In [ ]:
# Create synthetic training data for demonstration
def generate_synthetic_fire_data(num_samples=1000):
    """Generate synthetic fire detection training data"""
    # Simulate image features (in real scenario, use actual fire/non-fire images)
    data = []
    labels = []
    
    for i in range(num_samples):
        if i < num_samples // 2:
            # Fire images - higher red/orange values, specific patterns
            red_intensity = np.random.normal(200, 30)
            orange_intensity = np.random.normal(180, 25)
            brightness = np.random.normal(150, 20)
            motion_intensity = np.random.normal(80, 15)
            label = 1  # Fire
        else:
            # Non-fire images
            red_intensity = np.random.normal(100, 40)
            orange_intensity = np.random.normal(90, 35)
            brightness = np.random.normal(120, 30)
            motion_intensity = np.random.normal(30, 20)
            label = 0  # No fire
        
        # Create feature vector
        features = [
            red_intensity, orange_intensity, brightness, motion_intensity,
            np.random.normal(50, 10),  # Additional features
            np.random.normal(75, 15)
        ]
        
        data.append(features)
        labels.append(label)
    
    return np.array(data), np.array(labels)

def generate_synthetic_crowd_data(num_samples=1000):
    """Generate synthetic crowd density data"""
    data = []
    labels = []
    
    for i in range(num_samples):
        # Simulate crowd features
        people_count = np.random.randint(0, 200)
        area_coverage = np.random.uniform(0.1, 0.9)
        movement_speed = np.random.uniform(0, 5)
        noise_level = np.random.uniform(40, 100)
        
        # Calculate density level
        density = people_count * area_coverage / 100
        if density < 1.0:
            density_level = 0  # Low
        elif density < 2.5:
            density_level = 1  # Medium
        elif density < 4.0:
            density_level = 2  # High
        else:
            density_level = 3  # Critical
        
        features = [people_count, area_coverage, movement_speed, noise_level, density]
        data.append(features)
        labels.append(density_level)
    
    return np.array(data), np.array(labels)

def generate_synthetic_behavior_data(num_samples=1000):
    """Generate synthetic behavior analysis data"""
    data = []
    labels = []
    
    behavior_types = ['normal', 'suspicious', 'aggressive', 'panic']
    
    for i in range(num_samples):
        behavior = np.random.choice(behavior_types)
        
        if behavior == 'normal':
            motion_variance = np.random.normal(10, 3)
            audio_level = np.random.normal(60, 10)
            speed_changes = np.random.normal(2, 1)
        elif behavior == 'suspicious':
            motion_variance = np.random.normal(25, 5)
            audio_level = np.random.normal(50, 8)
            speed_changes = np.random.normal(8, 2)
        elif behavior == 'aggressive':
            motion_variance = np.random.normal(40, 8)
            audio_level = np.random.normal(85, 15)
            speed_changes = np.random.normal(15, 3)
        else:  # panic
            motion_variance = np.random.normal(60, 10)
            audio_level = np.random.normal(95, 20)
            speed_changes = np.random.normal(25, 5)
        
        features = [
            motion_variance, audio_level, speed_changes,
            np.random.normal(30, 10),  # Additional features
            np.random.normal(45, 12)
        ]
        
        data.append(features)
        labels.append(behavior)
    
    return np.array(data), np.array(labels)

# Generate synthetic datasets
print("Generating synthetic training data...")
fire_data, fire_labels = generate_synthetic_fire_data(2000)
crowd_data, crowd_labels = generate_synthetic_crowd_data(2000)
behavior_data, behavior_labels = generate_synthetic_behavior_data(2000)

print(f"Fire detection data: {fire_data.shape}, Labels: {fire_labels.shape}")
print(f"Crowd analysis data: {crowd_data.shape}, Labels: {crowd_labels.shape}")
print(f"Behavior analysis data: {behavior_data.shape}, Labels: {behavior_labels.shape}")
print("Synthetic data generated successfully!")

## 3. Fire Detection Model Training

In [ ]:
# Prepare fire detection data
X_fire_train, X_fire_test, y_fire_train, y_fire_test = train_test_split(
    fire_data, fire_labels, test_size=0.2, random_state=42, stratify=fire_labels
)

# Scale features
fire_scaler = StandardScaler()
X_fire_train_scaled = fire_scaler.fit_transform(X_fire_train)
X_fire_test_scaled = fire_scaler.transform(X_fire_test)

# Create and train fire detection model
def create_fire_detection_model(input_dim):
    model = models.Sequential([
        layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    
    return model

# Train fire detection model
print("Training fire detection model...")
fire_model = create_fire_detection_model(X_fire_train_scaled.shape[1])

# Callbacks
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001
)

# Train model
fire_history = fire_model.fit(
    X_fire_train_scaled, y_fire_train,
    validation_data=(X_fire_test_scaled, y_fire_test),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Evaluate model
fire_predictions = (fire_model.predict(X_fire_test_scaled) > 0.5).astype(int)
fire_accuracy = accuracy_score(y_fire_test, fire_predictions)

print(f"\nFire Detection Model Accuracy: {fire_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_fire_test, fire_predictions))

# Save model
fire_model.save('/content/fire_detection_model.h5')
print("Fire detection model saved!")

## 4. Crowd Density Analysis Model Training

In [ ]:
# Prepare crowd analysis data
X_crowd_train, X_crowd_test, y_crowd_train, y_crowd_test = train_test_split(
    crowd_data, crowd_labels, test_size=0.2, random_state=42, stratify=crowd_labels
)

# Scale features
crowd_scaler = StandardScaler()
X_crowd_train_scaled = crowd_scaler.fit_transform(X_crowd_train)
X_crowd_test_scaled = crowd_scaler.transform(X_crowd_test)

# Create and train crowd density model
def create_crowd_density_model(input_dim, num_classes):
    model = models.Sequential([
        layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Train crowd density model
print("Training crowd density analysis model...")
num_density_classes = len(np.unique(crowd_labels))
crowd_model = create_crowd_density_model(X_crowd_train_scaled.shape[1], num_density_classes)

# Train model
crowd_history = crowd_model.fit(
    X_crowd_train_scaled, y_crowd_train,
    validation_data=(X_crowd_test_scaled, y_crowd_test),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Evaluate model
crowd_predictions = np.argmax(crowd_model.predict(X_crowd_test_scaled), axis=1)
crowd_accuracy = accuracy_score(y_crowd_test, crowd_predictions)

print(f"\nCrowd Density Model Accuracy: {crowd_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_crowd_test, crowd_predictions))

# Save model
crowd_model.save('/content/crowd_density_model.h5')
print("Crowd density model saved!")

## 5. Behavior Analysis Model Training

In [ ]:
# Prepare behavior analysis data
# Encode labels
behavior_encoder = LabelEncoder()
behavior_labels_encoded = behavior_encoder.fit_transform(behavior_labels)

X_behavior_train, X_behavior_test, y_behavior_train, y_behavior_test = train_test_split(
    behavior_data, behavior_labels_encoded, test_size=0.2, random_state=42, stratify=behavior_labels_encoded
)

# Scale features
behavior_scaler = StandardScaler()
X_behavior_train_scaled = behavior_scaler.fit_transform(X_behavior_train)
X_behavior_test_scaled = behavior_scaler.transform(X_behavior_test)

# Train Random Forest for behavior analysis
print("Training behavior analysis model...")
behavior_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

behavior_model.fit(X_behavior_train_scaled, y_behavior_train)

# Evaluate model
behavior_predictions = behavior_model.predict(X_behavior_test_scaled)
behavior_accuracy = accuracy_score(y_behavior_test, behavior_predictions)

print(f"\nBehavior Analysis Model Accuracy: {behavior_accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(
    y_behavior_test, 
    behavior_predictions,
    target_names=behavior_encoder.classes_
))

# Feature importance
feature_names = ['Motion Variance', 'Audio Level', 'Speed Changes', 'Feature 4', 'Feature 5']
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': behavior_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(importance_df)

# Save model and encoders
import joblib
joblib.dump(behavior_model, '/content/behavior_analysis_model.pkl')
joblib.dump(behavior_encoder, '/content/behavior_label_encoder.pkl')
joblib.dump(behavior_scaler, '/content/behavior_scaler.pkl')
print("Behavior analysis model saved!")

## 6. Sensor Anomaly Detection Training

In [ ]:
# Generate synthetic sensor data
def generate_sensor_data(num_samples=2000):
    """Generate synthetic sensor readings"""
    data = []
    
    for i in range(num_samples):
        # Normal readings (90% of data)
        if i < num_samples * 0.9:
            temperature = np.random.normal(22, 3)  # Normal room temperature
            humidity = np.random.normal(45, 8)     # Normal humidity
            sound = np.random.normal(65, 10)       # Normal sound level
            motion = np.random.normal(20, 5)       # Normal motion
        else:
            # Anomalous readings (10% of data)
            temperature = np.random.choice([
                np.random.normal(45, 5),   # High temperature (fire)
                np.random.normal(-5, 2)    # Very low temperature
            ])
            humidity = np.random.normal(85, 10)    # High humidity
            sound = np.random.normal(110, 15)      # High sound (emergency)
            motion = np.random.normal(80, 20)      # High motion (panic)
        
        data.append([temperature, humidity, sound, motion])
    
    return np.array(data)

# Generate sensor data
sensor_data = generate_sensor_data(2000)

# Train anomaly detection models for each sensor type
sensor_types = ['temperature', 'humidity', 'sound', 'motion']
anomaly_models = {}
sensor_scalers = {}

for i, sensor_type in enumerate(sensor_types):
    print(f"Training anomaly detector for {sensor_type}...")
    
    # Extract sensor-specific data
    sensor_values = sensor_data[:, i].reshape(-1, 1)
    
    # Use only normal data for training (first 90%)
    normal_data = sensor_values[:int(len(sensor_values) * 0.9)]
    
    # Scale data
    scaler = StandardScaler()
    normal_data_scaled = scaler.fit_transform(normal_data)
    
    # Train Isolation Forest
    anomaly_model = IsolationForest(
        contamination=0.1,
        random_state=42,
        n_estimators=100
    )
    anomaly_model.fit(normal_data_scaled)
    
    # Test on all data
    all_data_scaled = scaler.transform(sensor_values)
    predictions = anomaly_model.predict(all_data_scaled)
    anomaly_scores = anomaly_model.decision_function(all_data_scaled)
    
    # Calculate metrics
    true_anomalies = np.concatenate([
        np.ones(int(len(sensor_values) * 0.9)),   # Normal = 1
        np.full(int(len(sensor_values) * 0.1), -1) # Anomaly = -1
    ])
    
    accuracy = accuracy_score(true_anomalies, predictions)
    print(f"{sensor_type} anomaly detection accuracy: {accuracy:.4f}")
    
    # Store models
    anomaly_models[sensor_type] = anomaly_model
    sensor_scalers[sensor_type] = scaler

# Save anomaly detection models
for sensor_type in sensor_types:
    joblib.dump(anomaly_models[sensor_type], f'/content/{sensor_type}_anomaly_model.pkl')
    joblib.dump(sensor_scalers[sensor_type], f'/content/{sensor_type}_scaler.pkl')

print("\nAll anomaly detection models saved!")

## 7. Model Performance Visualization

In [ ]:
# Plot training histories
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Fire detection model history
axes[0, 0].plot(fire_history.history['accuracy'], label='Training Accuracy')
axes[0, 0].plot(fire_history.history['val_accuracy'], label='Validation Accuracy')
axes[0, 0].set_title('Fire Detection Model - Accuracy')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True)

axes[0, 1].plot(fire_history.history['loss'], label='Training Loss')
axes[0, 1].plot(fire_history.history['val_loss'], label='Validation Loss')
axes[0, 1].set_title('Fire Detection Model - Loss')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Crowd density model history
axes[1, 0].plot(crowd_history.history['accuracy'], label='Training Accuracy')
axes[1, 0].plot(crowd_history.history['val_accuracy'], label='Validation Accuracy')
axes[1, 0].set_title('Crowd Density Model - Accuracy')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].legend()
axes[1, 0].grid(True)

axes[1, 1].plot(crowd_history.history['loss'], label='Training Loss')
axes[1, 1].plot(crowd_history.history['val_loss'], label='Validation Loss')
axes[1, 1].set_title('Crowd Density Model - Loss')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig('/content/model_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Confusion matrices
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Fire detection confusion matrix
fire_cm = confusion_matrix(y_fire_test, fire_predictions)
sns.heatmap(fire_cm, annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title('Fire Detection - Confusion Matrix')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# Crowd density confusion matrix
crowd_cm = confusion_matrix(y_crowd_test, crowd_predictions)
sns.heatmap(crowd_cm, annot=True, fmt='d', cmap='Greens', ax=axes[1])
axes[1].set_title('Crowd Density - Confusion Matrix')
axes[1].set_xlabel('Predicted')
axes[1].set_ylabel('Actual')

# Behavior analysis confusion matrix
behavior_cm = confusion_matrix(y_behavior_test, behavior_predictions)
sns.heatmap(behavior_cm, annot=True, fmt='d', cmap='Oranges', ax=axes[2])
axes[2].set_title('Behavior Analysis - Confusion Matrix')
axes[2].set_xlabel('Predicted')
axes[2].set_ylabel('Actual')

plt.tight_layout()
plt.savefig('/content/confusion_matrices.png', dpi=300, bbox_inches='tight')
plt.show()

print("Visualizations saved!")

## 8. Download Trained Models

In [ ]:
# Create a summary of all trained models
model_summary = {
    "training_date": datetime.now().isoformat(),
    "models": {
        "fire_detection": {
            "type": "neural_network",
            "accuracy": float(fire_accuracy),
            "file": "fire_detection_model.h5"
        },
        "crowd_density": {
            "type": "neural_network",
            "accuracy": float(crowd_accuracy),
            "file": "crowd_density_model.h5"
        },
        "behavior_analysis": {
            "type": "random_forest",
            "accuracy": float(behavior_accuracy),
            "files": [
                "behavior_analysis_model.pkl",
                "behavior_label_encoder.pkl",
                "behavior_scaler.pkl"
            ]
        },
        "anomaly_detection": {
            "type": "isolation_forest",
            "sensor_types": sensor_types,
            "files": [f"{sensor}_anomaly_model.pkl" for sensor in sensor_types] + 
                     [f"{sensor}_scaler.pkl" for sensor in sensor_types]
        }
    }
}

# Save model summary
with open('/content/model_summary.json', 'w') as f:
    json.dump(model_summary, f, indent=2)

# List all files to download
files_to_download = [
    '/content/fire_detection_model.h5',
    '/content/crowd_density_model.h5',
    '/content/behavior_analysis_model.pkl',
    '/content/behavior_label_encoder.pkl',
    '/content/behavior_scaler.pkl',
    '/content/model_summary.json',
    '/content/model_training_history.png',
    '/content/confusion_matrices.png'
]

# Add anomaly detection files
for sensor_type in sensor_types:
    files_to_download.extend([
        f'/content/{sensor_type}_anomaly_model.pkl',
        f'/content/{sensor_type}_scaler.pkl'
    ])

print("Files ready for download:")
for file_path in files_to_download:
    if os.path.exists(file_path):
        print(f"✓ {file_path}")
    else:
        print(f"✗ {file_path} (missing)")

print("\n" + "="*50)
print("TRAINING COMPLETE!")
print("="*50)
print(f"Fire Detection Accuracy: {fire_accuracy:.4f}")
print(f"Crowd Density Accuracy: {crowd_accuracy:.4f}")
print(f"Behavior Analysis Accuracy: {behavior_accuracy:.4f}")
print("\nDownload all files and place them in your project's 'data/models/' directory.")
print("Update the model paths in your configuration files accordingly.")